# 評価(Accuracy): 80%以上

## import関係

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import accuracy_score
from original_model import Net
import warnings
warnings.simplefilter('ignore')

## 関数関係

In [ ]:
def predict(img):
    #ファイル読み込み
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
    ])

    img = transform(img).unsqueeze(0)

     #予測
    output = model(img)
    softmax = nn.Softmax(dim=1)
    _, pred = torch.max(output, 1)
    ans = pred[0].item()
    return ans


## モデルをload

In [ ]:
model = Net().to(device)
model.load_state_dict(torch.load("../models/model_weight.pth"))
model.eval()

## 単体実行

In [ ]:
dog_img_path = "../TRAIN//犬/000001.jpg"
cat_img_path = "../TRAIN//猫/000001.jpg"
rabbit_img_path = "../TRAIN//ウサギ/000001.jpg"

dog_img = Image.open(dog_img_path)
cat_img = Image.open(cat_img_path)
rabbit_img = Image.open(rabbit_img_path)

print(predict(dog_img))
print(predict(cat_img))
print(predict(rabbit_img))

## TESTデータを予測

In [ ]:
annotation_df = pd.read_csv('../TEST/annotation.csv')
result_df = pd.DataFrame()

for index, row in annotation_df.iterrows():
    file_name = row['file_name']
    ans = row['annotation']

    if ans == "rabbit":
        annotation = 0
    elif ans == "Dog":
        annotation = 1
    else:
        annotation = 2

    img = Image.open(f'{file_name}')
    pred = predict(img)

    result_df = result_df.append({"file_name": file_name,
                                "ans": annotation,
                                "pred": pred},
                                ignore_index=True)

print("Finish!!")

## 評価

In [ ]:
y_true = result_df["ans"].values.tolist()
y_pred = result_df["pred"].values.tolist()

score = accuracy_score(y_true, y_pred) * 100

print(f"Accuracy: {score}")